In [1]:
import pandas as pd

# 0 Data preparation

In [2]:
df_journal = pd.read_csv('journal_aim2.csv')
df_journal.head(10)

,publication_title,title_id,Aim
0,advanced robotics,tadr20,advanced robotics (ar) is the international jo...
1,advances in building energy research,taer20,advances in building energy research (aber) ai...
2,advances in materials and processing technologies,tmpt20,the journal covers advancement in materials an...
3,aerosol science and technology,uast20,aerosol science and technology publishes theor...
4,architectural engineering and design management,taem20,"informative and accessible, this publication..."
5,architectural science review,tasr20,founded at the university of sydney in 1958 by...
6,assistive technology,uaty20,"assistive technology is an applied, scientific..."
7,australasian journal of engineering education,teen20,the australasian journal of engineering educat...
8,australasian journal of water resources,twar20,the australasian journal of water resources ( ...
9,australian journal of civil engineering,tcen20,the editorial policy of engineers australia’s ...


In [3]:
df_journal.describe()

,publication_title,title_id,Aim
count,226,226,226
unique,226,226,226
top,advanced robotics,tadr20,advanced robotics (ar) is the international jo...
freq,1,1,1


# 1 Get paper information

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import time

In [5]:
most_recent_link = 'https://www.tandfonline.com/action/showAxaArticles?journalCode='
most_cite_link = 'https://www.tandfonline.com/action/showMostCitedArticles?journalCode='

In [6]:
def get_paper_url(journal_url):

    list_url = []

    options = Options()
    options.add_argument('--headless')

    # download from https://chromedriver.storage.googleapis.com/index.html?path=100.0.4896.60/
    # for windows
    driver = webdriver.Chrome(executable_path='C:\ChromeDriver\chromedriver.exe', options=options)
    # for mac
    #driver = webdriver.Chrome(executable_path='/Users/athrunsun/Documents/Project/Interview/TandF2/chromedriver', options=options)

    driver.get(journal_url)
    time.sleep(3)

    #wait for a maximum of 60 seconds in this example
    #wait = WebDriverWait(driver, 10)
    #wait.until(lambda x: x.find_elements_by_xpath("//div[@class='art_title']/a[@href]"))

    # get links with 'art_title' 
    try:
        elements = driver.find_elements_by_xpath("//div[@class='art_title']/a[@href]")
        list_url = [elem.get_attribute('href') for elem in elements]
            # only keep links with doi 
        list_url_doi = [url_doi for url_doi in list_url if 'doi' in url_doi.lower()]
    except:
        list_url_doi = []
        

    #lnks=driver.find_elements_by_tag_name("a")
    #lnks=driver.find_elements_by_xpath("//a[@href]")
    

    #elements = driver.find_elements_by_css_selector(".art_title [href]")
    

    #list_url = [url.get_attribute('href') for url in lnks]
    #list_url = [elem.get_attribute("href") for elem in driver.find_elements_by_tag_name('a')]




    return list_url_doi
    

In [7]:
# single journal test

journal_id = 'tadr20'
journal_url = most_cite_link + journal_id

list_url = get_paper_url(journal_url)

print(list_url[0])

https://www.tandfonline.com/doi/full/10.1163/156855312X626343?src=


In [8]:
def get_title_keywords(paper_url):
    title = ''
    keywords = ''

    options = Options()
    options.add_argument('--headless')
    # download from https://chromedriver.storage.googleapis.com/index.html?path=100.0.4896.60/
    # for windows
    driver = webdriver.Chrome(executable_path='C:\ChromeDriver\chromedriver.exe', options=options)
    # for mac
    #driver = webdriver.Chrome(executable_path='/Users/athrunsun/Documents/Project/Interview/TandF2/chromedriver', options=options)

    driver.get(paper_url)
    #time.sleep(5)


    try:
    # get links with 'art_title' 
        title = driver.find_element_by_xpath("//span[@class='NLM_article-title hlFld-title']").text.lower()    
        elemt_keywords = driver.find_elements_by_xpath("//a [@class='kwd-btn keyword-click']")
        list_keywords = [elemt.text for elemt in elemt_keywords]
        keywords = ' '.join(list_keywords).lower()
    except:
        pass

    return title, keywords

In [9]:
# test single paper

paper_url = 'https://www.tandfonline.com/doi/full/10.1163/156855312X626343?src='

title, keywords = get_title_keywords(paper_url)
print(title)
print(keywords)



soft robot arm inspired by the octopus
octopus muscular hydrostats shape memory alloy soft robot


In [10]:
import gc 

In [11]:
col_name = ['paper_title', 'keywords', 'journal_name']
df_papers = pd.DataFrame([], columns=col_name) 
df_papers.to_csv('papers.csv',index=False)

for index, row in df_journal.iterrows():

    list_paper = []
    df_papers = pd.DataFrame([], columns=col_name)

    journal_title = row['publication_title']

    journal_id = row['title_id']
    journal_url = most_cite_link + journal_id
    list_url = get_paper_url(journal_url)

    # if index == 2: break # for debugging

    if list_url == []:        
        continue
    else:
        n_paper = 0
        for paper_url in list_url:
            title, keywords = get_title_keywords(paper_url)
            if (title != '' ) & (keywords != ''):
                list_paper.append([title, keywords, journal_title])
                n_paper += 1 

    
    df_papers = pd.DataFrame(list_paper, columns=col_name) 
    df_papers.to_csv('papers.csv',mode='a', index=False, header=False)
    gc.collect()
    print(index, ' ', journal_title,' get ', len(list_url),'papers in total')

 
  


0   advanced robotics  get  30 papers in total
1   advances in building energy research  get  30 papers in total
2   advances in materials and processing technologies  get  30 papers in total
3   aerosol science and technology  get  30 papers in total
4   architectural engineering and design management  get  30 papers in total
5   architectural science review  get  30 papers in total
6   assistive technology  get  30 papers in total
7   australasian journal of engineering education  get  30 papers in total
8   australasian journal of water resources  get  30 papers in total
9   australian journal of civil engineering  get  30 papers in total
10   australian journal of electrical and electronics engineering  get  30 papers in total
11   australian journal of mechanical engineering  get  30 papers in total
12   australian journal of multi-disciplinary engineering  get  30 papers in total
13   australian journal of structural engineering  get  30 papers in total
15   behaviour and informa

In [12]:
df_papers.head(10)

,paper_title,keywords,journal_name
0,thermo-hydro and thermo-hydro-mechanical wood ...,ageing bending densification heat treatment va...,wood material science and engineering
1,"a review on wood machining: characterization, ...",tool wear surface roughness cutting power dust...,wood material science and engineering
2,"understanding decay resistance, dimensional st...",acetylation acetyl content brown-rot fungus eq...,wood material science and engineering
3,chemical modification of wood: a short review,acetic anhydride acetylation chemical modifica...,wood material science and engineering
4,decay-influencing factors: a basis for service...,climate index direct factor durability estimat...,wood material science and engineering
5,consumers’ perceptions on the properties of wo...,multivariate methods market diffusion business...,wood material science and engineering
6,"relationship between wood porosity, wood densi...",density hardened wood impregnation mercury int...,wood material science and engineering
7,structural robustness and timber buildings – a...,robustness timber disproportionate collapse pr...,wood material science and engineering
8,effects of short-term thermomechanical densifi...,compression ratio mass loss short-term thermom...,wood material science and engineering
9,effect of thermomechanical densification on su...,surface roughness thermomechanical densificati...,wood material science and engineering
